Problem 1a) ADD Algorithm

In [2]:
import numpy as np

nodes = ['A','B','C','D','E','F']
f0 = [40, 100, 50, 115, 70, 100] # fixed costs
f=f0.copy()
h = [8, 10, 6, 5, 9, 10] # demand


c = {(0,0): 0 , (0,1): 10, (0,2): 16, (0,3): 10, (0,4): 12, (0,5): 16,
     (1,0): 10, (1,1): 0 , (1,2): 6 , (1,3): 7 , (1,4): 16, (1,5): 13,
     (2,0): 16, (2,1): 6 , (2,2): 0 , (2,3): 7 , (2,4): 16, (2,5): 9 ,
     (3,0): 10, (3,1): 7 , (3,2): 7 , (3,3): 0 , (3,4): 9 , (3,5): 6 ,
     (4,0): 12, (4,1): 16, (4,2): 16, (4,3): 9 , (4,4): 0 , (4,5): 9 ,
     (5,0): 16, (5,1): 13, (5,2): 9 , (5,3): 6 , (5,4): 9 , (5,5): 0 } # transportation distances * cost per unit demand (=1)


weighted_demand = np.zeros((6,6))
for i in range(6):
	for j in range(6):
		weighted_demand[i, j] = c[(i, j)] * h[i]
print(weighted_demand)

totals = [sum(weighted_demand[:,i]) * 1 + f[i] for i in range(6)]
print(totals)

[[  0.  80. 128.  80.  96. 128.]
 [100.   0.  60.  70. 160. 130.]
 [ 96.  36.   0.  42.  96.  54.]
 [ 50.  35.  35.   0.  45.  30.]
 [108. 144. 144.  81.   0.  81.]
 [160. 130.  90.  60.  90.   0.]]
[np.float64(554.0), np.float64(525.0), np.float64(507.0), np.float64(448.0), np.float64(557.0), np.float64(523.0)]


In [3]:
facilities_to_open = [nodes[np.argmin(totals)]]
print("Optimal location is at location:", nodes[np.argmin(totals)], "with cost:", min(totals))

Optimal location is at location: D with cost: 448.0


In [4]:
weighted_demand[np.argmin(totals),:] = 0

## Update all distances that are closer to the new facility than node i
weighted_demand[0,2] = h[0]*c[(0,3)]
weighted_demand[0,4] = h[0]*c[(0,3)]
weighted_demand[0,5] = h[0]*c[(0,3)]
weighted_demand[1,0] = h[1]*c[(1,3)]
weighted_demand[1,4] = h[1]*c[(1,3)]
weighted_demand[1,5] = h[1]*c[(1,3)]
weighted_demand[2,0] = h[2]*c[(2,3)]
weighted_demand[2,4] = h[2]*c[(2,3)]
weighted_demand[2,5] = h[2]*c[(2,3)]
weighted_demand[4,0] = h[4]*c[(4,3)]
weighted_demand[4,1] = h[4]*c[(4,3)]
weighted_demand[4,2] = h[4]*c[(4,3)]
weighted_demand[5,0] = h[5]*c[(5,3)]
weighted_demand[5,1] = h[5]*c[(5,3)]
weighted_demand[5,2] = h[5]*c[(5,3)]
weighted_demand[5,4] = h[5]*c[(5,3)]
print(weighted_demand)
for i in range(6):
    if nodes[i] == facilities_to_open[-1]:
        continue
    f[i] += f[np.argmin(totals)]
print(f)
totals = [sum(weighted_demand[:,i]) * 1 + f[i] for i in range(6)]
print(totals)

[[ 0. 80. 80. 80. 80. 80.]
 [70.  0. 60. 70. 70. 70.]
 [42. 36.  0. 42. 42. 42.]
 [ 0.  0.  0.  0.  0.  0.]
 [81. 81. 81. 81.  0. 81.]
 [60. 60. 60. 60. 60.  0.]]
[155, 215, 165, 115, 185, 215]
[np.float64(408.0), np.float64(472.0), np.float64(446.0), np.float64(448.0), np.float64(437.0), np.float64(488.0)]


In [5]:
facilities_to_open.append(nodes[np.argmin(totals)])
print("Next optimal location is at location:", nodes[np.argmin(totals)], "with cost:", min(totals))

Next optimal location is at location: A with cost: 408.0


In [6]:
weighted_demand[np.argmin(totals),:] = 0
## No distances to update this step
print(weighted_demand)
for i in range(6):
    if nodes[i] == facilities_to_open[-1]:
        continue
    f[i] += f[np.argmin(totals)]
print(f)
totals = [sum(weighted_demand[:,i]) * 1 + f[i] for i in range(6)]
print(totals)

[[ 0.  0.  0.  0.  0.  0.]
 [70.  0. 60. 70. 70. 70.]
 [42. 36.  0. 42. 42. 42.]
 [ 0.  0.  0.  0.  0.  0.]
 [81. 81. 81. 81.  0. 81.]
 [60. 60. 60. 60. 60.  0.]]
[155, 370, 320, 270, 340, 370]
[np.float64(408.0), np.float64(547.0), np.float64(521.0), np.float64(523.0), np.float64(512.0), np.float64(563.0)]


Now that we have no new facilities that give us a better objective value, we stop. So our optimal facility locations are 

In [7]:
print(facilities_to_open)

['D', 'A']


Locations D and A, with a total cost of $408.

Problem 1b) IP Solver

In [8]:
from gurobipy import Model, GRB, quicksum

I = [i for i in range(6)]
J = [j for j in range(6)]
IJ = [(i,j) for i in I for j in J]


model = Model("UFLP_1b")
x = model.addVars(J, vtype=GRB.BINARY)
y = model.addVars(IJ, vtype=GRB.CONTINUOUS, lb=0)

model.setObjective(quicksum(f[j]*x[j] for j in J) + quicksum(h[i]*c[i,j]*y[i,j] for (i,j) in IJ), GRB.MINIMIZE)
model.addConstrs((quicksum(y[i,j] for j in J) == 1 for i in I))
model.addConstrs((y[i,j] <= x[j] for (i,j) in IJ))
model.update()
model.optimize()

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 5 5625U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 42 rows, 42 columns and 108 nonzeros
Model fingerprint: 0x1a2fbf6f
Variable types: 36 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 42 rows, 42 columns, 108 nonzeros
Variable types: 36 continuous, 6 integer (6 binary)
Found heuristic solution: objective 1825.0000000
Found heuristic solution: objective 1750.0000000
Found heuristic solution: objective 1515.0000000

Root relaxation: objective 6.030000e+02, 23 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node

In [9]:
x

#y

{0: <gurobi.Var C0 (value 0.0)>,
 1: <gurobi.Var C1 (value -0.0)>,
 2: <gurobi.Var C2 (value -0.0)>,
 3: <gurobi.Var C3 (value 1.0)>,
 4: <gurobi.Var C4 (value -0.0)>,
 5: <gurobi.Var C5 (value -0.0)>}

The IP Solver finds a solution of opening facilities A, C, and E, with an objective cost of $345.

Problem 1c) Lagrangian Relaxation

In [51]:
# Lagrangian relaxation - run two iterations in this cell
avg_demand = np.mean(h)
avg_fixed = np.mean(f0)

# initialize lam as a dict of lists (one list per node)
lam = {node: [10*(avg_demand + avg_fixed)] for node in nodes}

alpha = 1.0
step_size = 2.0

I = [i for i in range(6)]
J = [j for j in range(6)]

def run_iteration(iter_num):
    global lam
    global alpha
    global step_size
    print("--- Iteration {} ---".format(iter_num))
    # recompute V, X, Y using latest lambdas
    Y = {(i,j): 0 for i in I for j in J}
    X = {j: 0 for j in J}
    V = {}
    for j in J:
        V[j] = f0[j] + sum(min(0, alpha * h[i] * c[(i,j)] - lam[nodes[i]][-1]) for i in I)
        X[j] = 1 if V[j] < 0 else 0
        for i in I:
            Y[(i,j)] = 1 if (X[j] == 1 and alpha * h[i] * c[(i,j)] - lam[nodes[i]][-1] < 0) else 0

    #print("X:", X)
    #print("Y:", Y)

    l_bound = sum(f0[j]*X[j] for j in J) + sum((alpha * h[i] * c[(i,j)] - lam[nodes[i]][-1])*Y[(i,j)] for i in I for j in J) + sum(lam[nodes[i]][-1] for i in I)
    print("LB:", l_bound)
    u_bound = sum(f0[j]*X[j] for j in J) + sum(h[i] * c[(i,j)] * Y[(i,j)] for i in I for j in J)
    print("UB:", u_bound)

    # update lambdas for this iteration (append)
    for i in I:
        node = nodes[i]
        violation = sum(Y[(i,j)] for j in J) - 1
        new_val = max(0, lam[node][-1] - step_size * violation)
        lam[node].append(new_val)

	# update step size
    step_size = (u_bound - l_bound) / sum((sum(Y[(i,j)] for j in J)**2) for i in I) if sum((sum(Y[(i,j)] for j in J)**2) for i in I) != 0 else step_size
    print("lambdas now:", {n: lam[n][-1] for n in nodes})
    return X, Y, l_bound, u_bound



for i in range(10):
	X, Y, lb, ub = run_iteration(i)

--- Iteration 0 ---
LB: -23036.000000000015
UB: 3114
lambdas now: {'A': np.float64(861.6666666666667), 'B': np.float64(861.6666666666667), 'C': np.float64(861.6666666666667), 'D': np.float64(861.6666666666667), 'E': np.float64(861.6666666666667), 'F': np.float64(861.6666666666667)}
--- Iteration 1 ---
LB: -22736.000000000015
UB: 3114
lambdas now: {'A': np.float64(256.3425925925924), 'B': np.float64(256.3425925925924), 'C': np.float64(256.3425925925924), 'D': np.float64(256.3425925925924), 'E': np.float64(256.3425925925924), 'F': np.float64(256.3425925925924)}
--- Iteration 2 ---
LB: -4576.2777777777665
UB: 3114
lambdas now: {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0}
--- Iteration 3 ---
LB: 0.0
UB: 0
lambdas now: {'A': np.float64(35.60313786008225), 'B': np.float64(35.60313786008225), 'C': np.float64(35.60313786008225), 'D': np.float64(35.60313786008225), 'E': np.float64(35.60313786008225), 'F': np.float64(35.60313786008225)}
--- Iteration 4 ---
LB: 213.6188271604935
UB: 0
lambdas

In [ ]:
print(X)
#print(Y)

{0: 1, 1: 0, 2: 0, 3: 0, 4: 1, 5: 0}
{(0, 0): 1, (0, 1): 0, (0, 2): 0, (0, 3): 0, (0, 4): 0, (0, 5): 0, (1, 0): 0, (1, 1): 0, (1, 2): 0, (1, 3): 0, (1, 4): 0, (1, 5): 0, (2, 0): 0, (2, 1): 0, (2, 2): 0, (2, 3): 0, (2, 4): 0, (2, 5): 0, (3, 0): 0, (3, 1): 0, (3, 2): 0, (3, 3): 0, (3, 4): 0, (3, 5): 0, (4, 0): 0, (4, 1): 0, (4, 2): 0, (4, 3): 0, (4, 4): 1, (4, 5): 0, (5, 0): 0, (5, 1): 0, (5, 2): 0, (5, 3): 0, (5, 4): 0, (5, 5): 0}
